# Importing ORACC Data from corpus.json
by Niek Veldhuis
UC Berkeley

March 2017

# TODO
* check that COFs are treated properly
* check that lines that continue into the next line (as in bilinguals) are captured completely. Such lines are indicated in the json by the the addition of 'l' (lower case L) to the reference (.ref).


# Introduction

Purpose of the code is to download [ORACC](http://oracc.org) JSON files that contain textual data and produce a `.csv` file with the relevant data for use in phylogenetics project. The JSON files contain all the transliteration and lemmatization data of an ORACC project (metadata are made available in a separate `.json` file). For an introduction to the various ORACC JSON files see the [ORACC Open Data](http://oracc.org/doc/opendata) page.

The resulting data file includes various elements of the ORACC data structure. The current code will output a file with the following fields: 

* id_line
* label
* lemma
* base
* extent
* scope

The fields `extent` and `scope` capture the number of missing lines or columns.

The selection of fields may be adjusted with standard `Pandas` functions.

## Notes
The current version of the script works with the `ijson` library. Documentation for [ijson](https://www.dataquest.io/blog/python-json-tutorial/), unfortunately, is extremely brief. Downloading the `.json` files from the ORACC server will take some time. The rest of the script is reasonably fast.

This notebook is written for **Python 3.5** with **Pandas 0.19** and **ijson 2.3**.

The notebook was written for the [Phylogenetics](https://github.com/ErinBecker/digital-humanities-phylogenetics) project with Erin Becker of [Data Carpentry](http://www.datacarpentry.org). The particular data selection and data manipulation performed in this notebook are inspired by the needs of that project (for instance, non-Sumerian words are filtered out). It should be fairly easy to adapt the notebook to the purposes of any other project that wishes to use [ORACC](http://oracc.org) data.

## Licensing
This notebook may be downloaded, used and adapted without any restrictions.

In [1]:
import pandas as pd   
import ijson
import urllib.request
import re
import tqdm

# Input List of Text IDs
Identify a list of text IDs (P, Q, and X numbers) in the directory `input`. The IDs are six-digit P, Q, or X numbers preceded by a project abbreviation in the format 'PROJECT/P######' or 'PROJECT/SUBPROJECT/Q######'. For example:
* dcclt/P117395
* etcsri/Q001203
* rinap/rinap1/Q003421

The list should be created with a flat text editor such as Textedit or Emacs, and the filename should end in `.txt`.

In [2]:
filename = input('Filename: ')

Filename: composites.txt


In [3]:
textids = '../data/text_ids/' + filename
with open(textids, 'r') as f:
    pqxnos = f.readlines()
pqxnos = [x.strip() for x in pqxnos]
prefix = [x[:-7] + 'corpusjson/' + x[-7:] for x in pqxnos]
pqxnos = [x[-7:] for x in pqxnos]
pqxnos[:5], prefix[:5]


(['Q000001', 'Q000040', 'Q000041', 'Q000042'],
 ['dcclt/corpusjson/Q000001',
  'dcclt/corpusjson/Q000040',
  'dcclt/corpusjson/Q000041',
  'dcclt/corpusjson/Q000042'])

# Parse
The function `oraccjasonparser()` takes one argument (the **url** of the `.json` file). It looks for the prefix `textid` to retrieve the six-digit P, Q, or X number of the text artifact. Parsing the file sequentially the code looks for the places where a line starts (`'.type' = 'line-start'`) and where a word starts (`'.node' = 'l'`, where `l` is for "lemma"). At each level the code will retrieve the relevant data and create a list where each entry is a dictionary that represents a single word. 

Words not only include lemmatized words, but also unlemmatized and unlemmatizable words (such as breaks).

The dictionary includes the keys `id_line` and `id_word` that allow the user to reassemble words and lines in order.

In [4]:
def oraccjasonparser(url):
    d = urllib.request.urlopen(url)
    parser = ijson.parse(d)
    word_l = []
    word_d = {}
    line_start = False
    word_start = False
    nonx = False
    for prefix, event, value in parser:
        if prefix == 'textid':
            id_text = value
#            print("parsing " + value)
        if prefix.endswith('.type'):
            if value == 'line-start':
                line_start = True
            else:
                line_start = False
        if line_start:
            if prefix.endswith('.ref') and not word_start:
                id_line = value # id_line is a reference number for a line
                                # that includes the id_text (e.g. P123456.49)
            if prefix.endswith('.label'):
                label = value   # label is a human-readable line number of the format
                                # o ii 24' (obverse column 2 line 24')
        if prefix.endswith('node'):
            if value == 'l':
                word_start = True
                if not word_d == {}:
                    word_l.append(word_d) # append the previous word to the list
                word_d = {}               # and start a new dictionary
                word_d['id_text'] = id_text # provide each word with appropriate 
                word_d['id_line'] = id_line # text and line-ID
                word_d['label'] = label     # and the line label.
            else:
                word_start = False
        if word_start:
            if prefix.endswith('.ref'):
                word_d['id_word'] = value
            if prefix.endswith('.sig'):
                word_d['signature'] = value
            if '.f.' in prefix:
                category = re.sub('.*\.', '', prefix) # get element after the last dot of the prefix
                word_d[category] = value # copy each element into the dictionary
        if prefix.endswith('.type'):
            if value == 'nonx':
                nonx = True
            else:
                nonx = False
        if nonx:                         # this captures so-called $-lines with information
            if prefix.endswith('.ref'):  # about number of broken lines/columns.
                id_line = value          # $-lines have their own id_line.
            if prefix.endswith('.strict'):
                if value == '1':           # select only 'strict' $ lines
                    if not word_d == {}:
                        word_l.append(word_d)
                    word_d = {}
                    word_d['id_line'] = id_line
                    word_d['id_text'] = id_text
                else:
                    nonx = False
            if prefix.endswith('.extent'): # capture the three elements of strict $ lines
                word_d['extent'] = value   # namely extent, scope, and state.
            if prefix.endswith('.scope'):
                word_d['scope'] = value
            if prefix.endswith('.state'):
                word_d['state'] = value

    word_l.append(word_d)  # make sure that the last word is captured, too.
    return(word_l) # return a list of dictionaries, where each entry (dictionary) in
                   # the list represents a word.

# Call the Parser Function for Each Textid

In [5]:
url_prefix = "http://oracc.museum.upenn.edu/"
word_l = []
for id_text in tqdm.tqdm(prefix):   # tqdm creates a progress bar
    try:
        url = url_prefix + id_text + '.json'
        word_l.extend(oraccjasonparser(url))
    except:
        print(url + ' not available') 

100%|██████████| 4/4 [00:07<00:00,  1.68s/it]


# Transform the Data into a DataFrame

In [6]:
words = pd.DataFrame(word_l)
words.head()

,base,cf,cont,epos,extent,form,gw,id_line,id_text,id_word,label,lang,morph,norm,norm0,pos,scope,sense,signature,state
0,udu,udu,NaN,N,NaN,udu,sheep,Q000001.1,Q000001,Q000001.1.1,1,sux,~,NaN,udu,N,NaN,sheep,@dcclt%sux:udu=udu[sheep//sheep]N'N$udu/udu#~,NaN
1,niga,niga,NaN,V/i,NaN,niga,fattened,Q000001.1,Q000001,Q000001.1.2,1,sux,~,NaN,niga,V/i,NaN,fattened,@dcclt%sux:niga=niga[fattened//fattened]V/i'V/...,NaN
2,udu,udu,NaN,N,NaN,udu,sheep,Q000001.2,Q000001,Q000001.2.1,2,sux,~,NaN,udu,N,NaN,sheep,@dcclt%sux:udu=udu[sheep//sheep]N'N$udu/udu#~,NaN
3,niga,niga,NaN,V/i,NaN,niga,fattened,Q000001.2,Q000001,Q000001.2.2,2,sux,~,NaN,niga,V/i,NaN,fattened,@dcclt%sux:niga=niga[fattened//fattened]V/i'V/...,NaN
4,sag₁₀,sag,NaN,V/i,NaN,sag₁₀,rare,Q000001.2,Q000001,Q000001.2.3,2,sux,~,NaN,sag,V/i,NaN,"(to be) fine, high quality","@dcclt%sux:sag₁₀=sag[rare//(to be) fine, high ...",NaN


# Remove Spaces and Commas from Guide Word and Sence
Spaces in Guide Word and Sense may cause trouble in computational methods in tokenization, or when saved in Comma Separated Values format. All spaces and commas are replaced by hyphens or nothing, respectively.

In [7]:
words = words.fillna('') # first replace Missing Values by empty string
words['sense'] = [x.replace(' ', '-') for x in words['sense']]
words['sense'] = [x.replace(',', '') for x in words['sense']]
words['gw'] = [x.replace(' ', '-') for x in words['gw']]
words['gw'] = [x.replace(',', '') for x in words['gw']]

The columns in the resulting DataFrame correspond to the elements of a full [ORACC](http://oracc.org) signature, plus information about text, line, and word ids:
* base (Sumerian only)
* cf (Citation Form)
* cont (continuation of the base; Sumerian only)
* epos (Effective Part of Speech)
* form (transliteration, omitting all flags such as indication of breakage)
* gw (Guide Word: main or first translation in standard dictionary)
* id_line (a line ID that begins with the six-digit P, Q, or X number of the text)
* id_text (six-digit P, Q, or X number)
* id_word (word ID that begins with the ID number of the line)
* label (traditional line number in the form o ii 2' (obverse column 2 line 2'), etc.)
* lang (language code, including sux, sux-x-emegir, sux-x-emesal, akk, akk-x-stdbab, etc)
* morph (Morphology; Sumerian only)
* norm (Normalization: Akkadian)
* norm0 (Normalization: Sumerian)
* pos (Part of Speech)
* sense (contextual meaning)
* signature (full ORACC signature)

Not all data elements (columns) are available for all words. Sumerian words never have a `norm`, Akkadian words do not have `norm0`, `base`, `cont`, or `morph`. Most data elements are only present when the word is lemmatized; only `lang`, `form`, `pos`, `id_word`, `id_line`, and `id_text` should always be there. An unlemmatized word has `pos` 'X' (for unknown). Broken words have `pos` 'u' (for 'unlemmatizable).

In [8]:
#words[words['label'] == '']

# Manipulate
The columns may be manipulated with standard Pandas methods to create the desired output. By way of example, the following code will create a column `lemma` with the format **cf[gw]pos** (for instance **lugal[king]N**). For words that have no lemmatization `lemma` equals `form`. Only Sumerian words are allowed (and thus `lang` can be omitted) and in addition to the column `lemma` the column `base` is preserved; words that have no lemmatization take `form` as their base. Words and bases are concatenated to lines.

## Remove  non-Sumerian words

In [9]:
lang = ['sux', ''] # note that 'lang' is empty in entries that indicate damage
words = words.loc[words['lang'].str[:3].isin(lang)].reset_index()

## Create Lemma Column and Adjust Base

In [10]:
words['lemma'] = words['cf'] # first element of lemma is the citation form
words['lemma'] = [words['lemma'][i] + '[' + words['gw'][i] 
                     + ']' + words['pos'][i] 
                     if not words['lemma'][i] == '' 
                     else words['form'][i] +'[NA]NA' for i in range(len(words))]
words['lemma'] = [lemma if not lemma == '[NA]NA' else '' for lemma in words['lemma'] ]
words['base'] = [words['base'][i] if not words['base'][i] == '' 
                 or words['label'][i] == '' else words['form'][i] 
                 for i in range(len(words))]
#lemmas = words[['lemma', 'id_text', 'id_line', 'id_word', 'label']]#, 'extent', 'scope']]
lemmas = words[['lemma', 'base', 'id_text', 'id_line', 'id_word', 'label', 'extent', 'scope']]
lemmas.head()

,lemma,base,id_text,id_line,id_word,label,extent,scope
0,udu[sheep]N,udu,Q000001,Q000001.1,Q000001.1.1,1,,
1,niga[fattened]V/i,niga,Q000001,Q000001.1,Q000001.1.2,1,,
2,udu[sheep]N,udu,Q000001,Q000001.2,Q000001.2.1,2,,
3,niga[fattened]V/i,niga,Q000001,Q000001.2,Q000001.2.2,2,,
4,sag[rare]V/i,sag₁₀,Q000001,Q000001.2,Q000001.2.3,2,,


## Group by Line

In [11]:
lines = words.groupby([words['id_line'], words['label']]).agg({
        'lemma': ' '.join,
        'base': ' '.join,
        'extent': ''.join, 
        'scope': ''.join
    }).reset_index()
        

In [12]:
df = lines[['id_line', 'label', 'lemma', 'base', 'extent', 'scope']]
df

,id_line,label,lemma,base,extent,scope
0,Q000001.1,1,udu[sheep]N niga[fattened]V/i,udu niga,,
1,Q000001.10,10,udu[sheep]N aslum[sheep]N niga[fattened]V/i sa...,udu aslumₓ(|A.LUM|) niga sag₁₀,,
2,Q000001.100,100,udu[sheep]N mašanše[animals]N,udu maš₂-anše,,
3,Q000001.101,101,udu[sheep]N babbar[white]V/i,udu babbar,,
4,Q000001.102,102,udu[sheep]N giggi[black]V/i,udu giggi,,
5,Q000001.103,103,udu[sheep]N su[red]V/i,udu su₄,,
6,Q000001.104,104,udu[sheep]N sissig[green]V/i,udu sig₇-sig₇,,
7,Q000001.105,105,udu[sheep]N gunu[speckled]V/i,udu gun₃,,
8,Q000001.106,106,udu[sheep]N mu[sound]V/i ša[cvve]V/t,udu mu₇ ša₄,,
9,Q000001.107,107,u[ewe]N,u₈,,


In [13]:
df['id_text'] = df['id_line'].str[:7]
df['line'] = [int(re.sub('.+\.', '', line)) for line in df['id_line']] #create a line number for sorting
df = df.sort_values(['id_text', 'line']).reset_index(drop=True)
df

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,id_line,label,lemma,base,extent,scope,id_text,line
0,Q000001.1,1,udu[sheep]N niga[fattened]V/i,udu niga,,,Q000001,1
1,Q000001.2,2,udu[sheep]N niga[fattened]V/i sag[rare]V/i,udu niga sag₁₀,,,Q000001,2
2,Q000001.3,3,udu[sheep]N niga[fattened]V/i ŋirgul[~wool]N,udu niga ŋir₂-gu-la,,,Q000001,3
3,Q000001.4,4,udu[sheep]N nita[male]N,udu nita₂,,,Q000001,4
4,Q000001.5,5,udu[sheep]N nita[male]N ŋešdu[breeder]N,udu nita₂ ŋeš-du₃-a,,,Q000001,5
5,Q000001.6,6,udu[sheep]N u[grass]N,udu u₂,,,Q000001,6
6,Q000001.7,7,udu[sheep]N utuwa[stud]N,udu utua(|DAG.KISIM₅×UŠ|),,,Q000001,7
7,Q000001.8,8,udu[sheep]N aslum[sheep]N,udu aslumₓ(|A.LUM|),,,Q000001,8
8,Q000001.9,9,udu[sheep]N aslum[sheep]N niga[fattened]V/i,udu aslumₓ(|A.LUM|) niga,,,Q000001,9
9,Q000001.10,10,udu[sheep]N aslum[sheep]N niga[fattened]V/i sa...,udu aslumₓ(|A.LUM|) niga sag₁₀,,,Q000001,10


## Save in CSV Format

In [14]:
filename = filename[:-4]
with open('../data/' + filename + '.csv', 'w') as w:
    df.to_csv(w, encoding='utf8')